In [8]:
import os
from os.path import join as opj
import numpy as np
import random

import open3d as o3d
from open3d.web_visualizer import draw
import matplotlib.pyplot as plt

from utils import *
from dataset import *

import pickle as pkl
data_root = 'data_root/3DGraspAff'
train_or_test = 'train' # 'train' or 'test'
aff_path = opj(data_root, 'scaled_anntd_remapped_full_shape_'+train_or_test+'_data.pkl')
with open(aff_path, 'rb') as f:
    aff_dataset = pkl.load(f)
print("Loaded dataset with {} objects".format(aff_dataset['num_objects']))
print("Num of semantic classes: ", len(aff_dataset["semantic_classes"]))
print("Semantic classes: ", aff_dataset["semantic_classes"])
print("Num of affordances: ", len(aff_dataset["affordance_classes"]))
print("All affordances: ", aff_dataset["affordance_classes"])

Loaded dataset with 1096 objects
Num of semantic classes:  7
Semantic classes:  ['Knife', 'Bag', 'Bottle', 'Scissors', 'Mug', 'Earphone', 'Hat']
Num of affordances:  7
All affordances:  ['handover', 'cut', 'stab', 'lift', 'wrap', 'pour', 'wear']


In [9]:
# Filter out thin objects from the dataset dictionary
o3d.utility.set_verbosity_level(o3d.utility.VerbosityLevel.Error)

SHAPENET_SAVED_PATH = data_root + '/obj_meshes/'+train_or_test+'/ShapeNet'
PARTNET_SAVED_PATH  = data_root + '/obj_meshes/'+train_or_test+'/PartNet'

min_xyz_width = 0.008 # atleast this width in each dimension

# loop through the dataset
for sem_classs in aff_dataset['semantic_classes']:
    shape_ids = list(aff_dataset['data'][sem_classs].keys())
    start_id = 0
    end_id = len(shape_ids)
    # end_id = random.randint(0, len(shape_ids))
    for shape_id in shape_ids[start_id:end_id]:
        print("Processing shape_id: ", shape_id, " for class: ", sem_classs)
        shapeid_path = opj(SHAPENET_SAVED_PATH, shape_id)
        obj_path = opj(shapeid_path, 'models', 'model_normalized.obj')
        # check if obj_path exists
        if not os.path.exists(obj_path):
            print("ShapeNet category not found, try PartNet?")
            partnet_anno_id = aff_dataset['data'][sem_classs][shape_id]['partnet_anno_id']
            obj_path = opj(PARTNET_SAVED_PATH, partnet_anno_id, 'model_normalized.obj')

        # try load with open3d
        obj_mesh = o3d.io.read_triangle_mesh(obj_path)
        
        # get max extents of the mesh
        xyz_widths = np.max(obj_mesh.vertices, axis=0) - np.min(obj_mesh.vertices, axis=0)
        
        # check if the mesh is too thin
        if np.min(xyz_widths) < min_xyz_width:
            print("Mesh too thin, removing shape_id: ", shape_id)
            # remove the shape_id from the dataset
            del aff_dataset['data'][sem_classs][shape_id]

Processing shape_id:  2ec5e09eaa1614bb41832a6387f5a648  for class:  Scissors
ShapeNet category not found, try PartNet?
Mesh too thin, removing shape_id:  2ec5e09eaa1614bb41832a6387f5a648
Processing shape_id:  e25c78d9-3574-4d6e-9cb2-f2905b19d51f  for class:  Scissors
ShapeNet category not found, try PartNet?
Processing shape_id:  297c4f8e6d89d0b33d35434498261b  for class:  Scissors
ShapeNet category not found, try PartNet?
Mesh too thin, removing shape_id:  297c4f8e6d89d0b33d35434498261b
Processing shape_id:  d5189176b6a6677bdf053ccc645157ff  for class:  Scissors
ShapeNet category not found, try PartNet?
Processing shape_id:  f5429496147bfee91683a1f2bb7ab319  for class:  Scissors
ShapeNet category not found, try PartNet?
Processing shape_id:  u77c517dd-333f-49aa-be71-e66bcb785c40  for class:  Scissors
ShapeNet category not found, try PartNet?
Processing shape_id:  31a4143c-fbd9-46d0-acf5-844893017852  for class:  Scissors
ShapeNet category not found, try PartNet?
Processing shape_id:  

In [10]:
# # Debug viz
# # load soccer ball
# soccer_ball = o3d.io.read_triangle_mesh('/home/sjauhri/IAS_WS/potato-net/GIGA-TSDF/GIGA-6DoF/data/urdfs/pile/test/soccer_ball_poisson_000_visual.obj')
# # # paint
# soccer_ball.paint_uniform_color([1, 0, 0])
# axes_mesh = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=[0, 0, 0])
# draw([obj_mesh, soccer_ball, axes_mesh])
# print(np.min(obj_mesh.vertices, axis=0))

In [11]:
# Save updated dataset
new_aff_path = opj(data_root, 'filt_scaled_anntd_remapped_full_shape_'+train_or_test+'_data.pkl')
with open(new_aff_path, 'wb') as f:
    pkl.dump(aff_dataset, f)